<script type="text/javascript"
  src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.0/MathJax.js?config=TeX-AMS_CHTML">
</script>
<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    tex2jax: {
      inlineMath: [['$','$'], ['\\(','\\)']],
      processEscapes: true},
      jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
      extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
      TeX: {
      extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
      equationNumbers: {
      autoNumber: "AMS"
      }
    }
  });
</script>

Jakub Musiał

# **AOD - lab2**

## **Exercise 1 - Jet fuel purchase**

Knowing the airports' fuel demands and comapnies' production capabilities and delivery costs, determine:
* The purchase plan and delivery costs which minimize the total cost
* The minimal total fuel delivery cost for all airports
* Wheter all companies will deliver the fuel
* Whether the fuel production capabilities are fully utilized

In [1]:
using JuMP
using GLPK

import JSON

#### **Data and utils**

In [2]:
# Data initialization
data_general = JSON.parse(read("data.json", String))
# Extraxt exercise 1 data from the general dictionary
data = data_general["ex1"]

Dict{String, Any} with 3 entries:
  "max_fuel"    => Dict{String, Any}("c2"=>550000, "c1"=>275000, "c3"=>660000)
  "cost"        => Dict{String, Any}("c2"=>Dict{String, Any}("a3"=>12, "a2"=>11…
  "fuel_demand" => Dict{String, Any}("a3"=>330000, "a2"=>220000, "a1"=>110000, …

In [3]:
# Extract company and airport names
companies = sort(collect(keys(data["max_fuel"])))
airports = sort(collect(keys(data["fuel_demand"])))
companies, airports

(["c1", "c2", "c3"], ["a1", "a2", "a3", "a4"])

In [4]:
# Delivery cost per gallon function
cpg(company::String, airport::String) = data["cost"][company][airport]

cpg (generic function with 1 method)

#### **Build the model**

Notation:
* $A = \text{airporits}$
* $C = \text{companies}$

In [5]:
model = Model()
set_optimizer(model, GLPK.Optimizer)

**Predictor variables:**

$x_{c, a}$ where $(c, a) \text{ } \epsilon \text{ } C \times A$ - fuel ammount delivered by company $c$ to airport $a$

In [6]:
@variable(model, x[companies, airports] >= 0)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["c1", "c2", "c3"]
    Dimension 2, ["a1", "a2", "a3", "a4"]
And data, a 3×4 Matrix{VariableRef}:
 x[c1,a1]  x[c1,a2]  x[c1,a3]  x[c1,a4]
 x[c2,a1]  x[c2,a2]  x[c2,a3]  x[c2,a4]
 x[c3,a1]  x[c3,a2]  x[c3,a3]  x[c3,a4]

**Constraints:** 
* Fuel ammounts must be non-negative

  $(\forall{c \epsilon C})(\forall{a \epsilon A})(x_{c, a} \geq 0)$

* Fuel companies cannot deliver more fuel than they can produce
  
  $(\forall{c \epsilon C}) (\sum_{a \epsilon A} x_{c, a} \leq max\_production(c))$

* Airports must receive exactly the ammount of fuel that they have a demand for

  $(\forall{a \epsilon A}) (\sum_{c \epsilon C} x_{c, a} = fuel\_demand(a))$

In [7]:
@constraints(model, begin
    [c in companies], sum(x[c, :]) <= data["max_fuel"][c]
    [a in airports], sum(x[:, a]) == data["fuel_demand"][a]
end)

(1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["c1", "c2", "c3"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[c1,a1] + x[c1,a2] + x[c1,a3] + x[c1,a4] <= 275000.0
 x[c2,a1] + x[c2,a2] + x[c2,a3] + x[c2,a4] <= 550000.0
 x[c3,a1] + x[c3,a2] + x[c3,a3] + x[c3,a4] <= 660000.0, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["a1", "a2", "a3", "a4"]
And data, a 4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64

**Objective:** minimal cost of the required fuel purchase

$min(\sum{a \epsilon A} \sum_{c \epsilon C} (cost\_per\_gallon(c, a) * x_{c, a}))$

In [8]:
@objective(
    model, 
    Min, 
    sum(cpg(c, a) * x[c, a] for c in companies, a in airports)
)

10 x[c1,a1] + 10 x[c1,a2] + 9 x[c1,a3] + 11 x[c1,a4] + 7 x[c2,a1] + 11 x[c2,a2] + 12 x[c2,a3] + 13 x[c2,a4] + 8 x[c3,a1] + 14 x[c3,a2] + 4 x[c3,a3] + 9 x[c3,a4]

In [9]:
# Optimize the model
optimize!(model)
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 8.52500e+06
  Objective bound    : -Inf
  Dual objective value : 8.52500e+06

* Work counters
  Solve time (sec)   : 0.00000e+00


#### **Results**

* The purchase plan and delivery costs:

In [11]:
println("Purchase plans:")
for a in airports
    println("Airport ", a, ":")
    for c in companies
        v = value(x[c, a])
        println("\t* ", c, " - ", v, " gallons (price: ", v * cpg(c, a), ")")
    end
end

Purchase plans:
Airport a1:
	* c1 - 0.0 gallons (price: 0.0)
	* c2 - 110000.0 gallons (price: 770000.0)
	* c3 - 0.0 gallons (price: 0.0)
Airport a2:
	* c1 - 165000.0 gallons (price: 1.65e6)
	* c2 - 55000.0 gallons (price: 605000.0)
	* c3 - 0.0 gallons (price: 0.0)
Airport a3:
	* c1 - 0.0 gallons (price: 0.0)
	* c2 - 0.0 gallons (price: 0.0)
	* c3 - 330000.0 gallons (price: 1.32e6)
Airport a4:
	* c1 - 110000.0 gallons (price: 1.21e6)
	* c2 - 0.0 gallons (price: 0.0)
	* c3 - 330000.0 gallons (price: 2.97e6)


<br />

* Minimal total fuel delivery cost for all airports:

    $cost_{min} = \sum_{a \epsilon A} \sum_{c \epsilon C} x[c, a] * cost\_per\_gallon(c, a)$

In [12]:
sum(value(x[c, a]) * cpg(c, a) for c in companies, a in airports)

8.525e6

* All companies will deliver the fuel:

    $
    all = 
        \begin{cases}
        true & \text{: } (\forall{c \epsilon C})((\exists{a \epsilon A})(x[c, a] > 0))\\
        false & \text{: else}
        \end{cases}
    $

In [13]:
all(any(value(x[c, a]) > 0 for a in airports) for c in companies)

true

* Fuel production capabilities are fully utilized:

    $
    full(c) = 
        \begin{cases}
        true & \text{: } \sum_{a \epsilon A} x[c, a] = max\_production(c)\\
        false & \text{: else}
        \end{cases}
    $

In [15]:
println("Fully utlizied:")
for c in companies
    println(c, ": ", sum(x[c, :]) == data["max_fuel"][c])
end

Fully utlizied:
c1: false
c2: false
c3: false
